In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import random
import time

In [43]:
URL = 'https://www.amazon.in/s?k=smart+lock&page=7&crid=3MNJ4J5A0A3FY&qid=1730092715&sprefix=smart+lock%2Caps%2C209&ref=sr_pg_7'
API_KEY = '53fde5daee9e3fd8b128656faaf77e49'

# ScraperAPI setup
api_url = f'http://api.scraperapi.com?api_key={API_KEY}&url={URL}'

In [44]:
# HTTP Request
webpage = requests.get(api_url)
# Soup Object containiang all data
soup = BeautifulSoup(webpage.content, "html.parser")

In [46]:
links = soup.find_all("a", attrs={'class': 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

In [47]:
print(len(links))

36


In [48]:
# Initialize the list to store product data
product_data = []

# Function to extract Best Seller Rank
def fetch_best_seller_rank(soup):
    try:
        table = soup.find("table", {"id": "productDetails_detailBullets_sections1"})
        rows = table.find_all("tr") if table else []

        for row in rows:
            header = row.find("th", class_="a-color-secondary a-size-base prodDetSectionEntry")
            value = row.find("span")

            if header and "Best Sellers Rank" in header.text:
                ranks = [rank.strip() for rank in value.text.split("\n") if rank.strip()]
                return ranks[0] if ranks else "N/A"  # Select the primary rank
    except AttributeError:
        return "N/A"
    return "N/A"

for i in range(len(links)):
    try:
        link = links[i].get('href')
        product_link_full = "https://amazon.in" + link
        product_url = f'http://api.scraperapi.com?api_key={API_KEY}&url={product_link_full}'

        # Request product page
        product_webpage = requests.get(product_url)
        soup = BeautifulSoup(product_webpage.content, "html.parser")

        # Extract details including Best Seller Rank
        details = {
            'Product URL': product_link_full,
            'Product Title': soup.find("span", attrs={"id": 'productTitle'}).text.strip() if soup.find("span", attrs={"id": 'productTitle'}) else "N/A",
            'Brand': soup.find("span", attrs={"class": 'a-size-base po-break-word'}).text.strip() if soup.find("span", attrs={"class": 'a-size-base po-break-word'}) else "N/A",
            'Price': soup.find("span", attrs={"class": 'a-price aok-align-center reinventPricePriceToPayMargin priceToPay'}).text.strip() if soup.find("span", attrs={"class": 'a-price aok-align-center reinventPricePriceToPayMargin priceToPay'}) else "N/A",
            'Rating': soup.find("span", attrs={"class": 'a-size-base a-color-base'}).text.strip() if soup.find("span", attrs={"class": 'a-size-base a-color-base'}) else "N/A",
            'Rating Count': soup.find("span", attrs={"id": 'acrCustomerReviewText'}).text.strip() if soup.find("span", attrs={"id": 'acrCustomerReviewText'}) else "N/A",
            'Best Seller Rank': fetch_best_seller_rank(soup)  # Fetch and add Best Seller Rank
        }

        # Append extracted details to product data
        product_data.append(details)

    except Exception as e:
        print(f"Error processing product at index {i}: {e}")

# Save data to a CSV file
df = pd.DataFrame(product_data)
df.to_csv("/content/csvfiles/product_data7.csv", index=False)
print("Data saved to product_data.csv")


Data saved to product_data.csv


In [1]:
import pandas as pd
import glob

# Define the path to your CSV files
path = '/Users/saiganeshreddykodekandla/Documents/projects/circuitTechnologies/circuitTechnologiesAmazon/aDataScrapingAmazon/csvFiles/*.csv'
# Use glob to get all CSV files in the folder
all_files = glob.glob(path)

# Create an empty list to hold the DataFrames
dataframes = []

# Loop through each file and read it into a DataFrame
for file in all_files:
    df = pd.read_csv(file)
    dataframes.append(df)

# Concatenate all DataFrames into a single DataFrame
merged_data = pd.concat(dataframes, ignore_index=True)

# Save the merged DataFrame to a new CSV file
merged_data.to_csv('/Users/saiganeshreddykodekandla/Documents/projects/circuitTechnologies/circuitTechnologiesAmazon/csv_files/rawDataAmazonScrap.csv', index=False)